In [ ]:
# 📦 Install necessary libraries if not already installed (for Colab)
!pip install -q transformers ipywidgets


In [4]:
# Requirements:
# pip install transformers torch

from transformers import pipeline, set_seed
import torch

# Optional: Set seed for reproducibility
set_seed(42)

# === Step 1: Model Selection ===
# You can change the model here (smaller models need less RAM)
model_id = 'EleutherAI/gpt-neo-2.7B'  # or 'distilgpt2', 'gpt2', etc.

# === Step 2: Input Topic ===
prompt = input("📝 Enter your topic or prompt: ")

# === Step 3: Device Setup ===
device = 0 if torch.cuda.is_available() else -1
print(f"\n🚀 Loading model `{model_id}` on {'CUDA' if device == 0 else 'CPU'}...")

# === Step 4: Load Model and Generate Text ===
generator = pipeline("text-generation", model=model_id, device=device)

result = generator(
    prompt,
    max_length=250,
    do_sample=True,
    temperature=0.8,
    top_k=50,
    top_p=0.95,
    repetition_penalty=1.5,
    truncation=True,
    pad_token_id=50256
)[0]["generated_text"]

# === Step 5: Display Result ===
print("\n===========================\n")
print("### 🧠 Generated Text:\n")
print(result)



📝 Enter your topic or prompt: Impact of AI in Humans

🚀 Loading model `EleutherAI/gpt-neo-2.7B` on CUDA...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/10.7G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Device set to use cuda:0
Both `max_new_tokens` (=256) and `max_length`(=250) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)




### 🧠 Generated Text:

Impact of AI in Humans

There are a lot of people working on the development of artificial intelligence (AI). It is not a new thing for humans to do. Artificial intelligence is what is made from the human brain. AI was first seen by Alan Turing when he used an AI to write the world’s first computer program. In other words, AI is the process of creating machines that can think, make decisions, predict, and learn. AI is one of the fastest developing fields of science.

This article will explore some of the benefits of AI in humans. One of the benefits of AI is that it can help us to solve problems that were impossible before. This is because AI can analyze large amounts of data, which helps to find solutions to complex problems. In addition, AI can also improve our decision making process. AI can help us to find more accurate answers to problems.

AI can also provide us with new tools that can help us to develop new products. AI can also help us to develop new wa